In [42]:
import openpyxl
import pandas as pd
from natsort import index_natsorted
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [43]:
be = pd.read_excel("Spielwiese_Matchen_III_Drucke_1501-1560.xlsm", sheet_name="III Drucke 1501-1560", dtype="string")
iii = pd.read_csv("../abzug/iii.csv", dtype="string")

In [21]:
be

,Lfd Nr.,AKZ,IDN,Link zum Portal,Signatur,Provenienzmerkmal,Material,Format,Öffnungswinkel,Einschränkungen,...,Risse am Rand,Fehl-stellen im BB\n(groß),Insekten-fraß (stark),Falten / Knicke,saures /\nbrüchiges Papier,Tinten-/ Farbfraß (akut),Register-marken fragil,Klebe-\nstreifen ablösen,Rest.-Aufwand Buchblock\n(in Std.),Anmerkungen für die Restaurierung am Buchblock
0,1,L-1502-315318171,1066859469,<NA>,"III 2, 1",X,Ledereinband,bis 25 cm,"80° bis 110°, einseitig digitalisierbar?","hohler Rücken, welliger Buchblock",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,L-1534-315317728,1066859000,<NA>,"III 2, 2",X,"Ledereinband, Schließen, erhabene Buchbeschläge",bis 35 cm,180°,"hohler Rücken, welliger Buchblock",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,3,L-1540-180391380,1003653014,<NA>,"III 2, 3",X,Pergamentband,bis 25 cm,"80° bis 110°, einseitig digitalisierbar?","hohler Rücken, welliger Buchblock, Einband mit...",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,4,L-1535-168291959,999397737,<NA>,"III 2, 4",<NA>,Halbpergamentband,bis 25 cm,"80° bis 110°, einseitig digitalisierbar?","hohler Rücken, welliger Buchblock",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,5,L-1555-175067589,1001530470,<NA>,"III 2, 5",X,Pergamentband,bis 25 cm,"80° bis 110°, einseitig digitalisierbar?","hohler Rücken, welliger Buchblock",...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1216,L-1520-315469781,1066942145,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 60, 21",<NA>,<NA>,bis 25 cm,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1231,1217,L-1524-678834245,1211124975,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 60, 21 (Angebundenes Werk)",<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1232,<NA>,<NA>,<NA>,<NA>,Testsignatur 2022,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1233,1218,L-1525-158505301,995010374,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 69, 32",<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


# neuer Ansatz: merge

In [63]:
df = be.merge(iii, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

# idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
df.loc[df["IDN"].isna(), "IDN"] = df.idn
df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

# wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
df.loc[df["idn"].notna(), "digi"] = True
df.loc[df["idn"].isna(), "digi"] = False
df.fillna('', inplace=True)
df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))

# die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
spaltenliste = list(df.columns)[:-22]
spaltenliste.insert(0,"digi")
spaltenliste.insert(4, "bbg")
spaltenliste.insert(7, "signatur_g")

df.to_excel("iii-merge-test.xlsx", index=False, columns=spaltenliste)

In [64]:
df[spaltenliste]

,digi,Lfd Nr.,AKZ,IDN,bbg,Link zum Portal,Signatur,signatur_g,Provenienzmerkmal,Material,Format,Öffnungswinkel,Einschränkungen,Glasplatte,Verpackung,Verpackung austauschen,Schadensklasse,"notwendige Reparatur(en) vor der Digitalisierung, notwendige Reparatur(en) vor der Digitalisierung",Bemerkungen,Fragen/ Hinweise\nan DBSM,Fragen an M. Steinberg,nicht\nam Stand-ort,Größe ÜF\n(BxH),Breite\n(nur Ausreißer),Dicke\n(>12 cm),12° Format\n(<15 cm),Einband-\nart,Einband über-\nformt (ganz od. teilweise),Buch bereits restau-riert,hohler/\nfester Rücken (mit Einlage/\nVergoldung?),Steh-\nkanten\n(bei Perg.),Leder pudert ab/roter Zerfall (extrem),Einband stark defor-miert,Be-schläge bes. auftra-gend,Buch-schließe steif,Buch-block Pa./Perg.,saures Füll-material,Register-marken,seitliche Heftung,Buch-block sehr wellig,"Buch-block neigt zum ""Bauch""",ge-schloss-ene Lagen,Falttafeln,Größe Buch+\nFalttafeln (BxH),Original-grafik,Kolorier-ung / Buch-malerei / Initialen / Rubri-kation,berühr-ungsfreie Digit.,Schrift weit bis in den Falz (Bund-steg in mm) Text-verlust,nicht digitali-sierbar wegen Bund-steg (vorraus-sichtlich),max. Öffnungs-winkel,Digit. mit Begleit-ung,Rest.-Bericht einge-klebt,Blatt mit Notizen zum Buch eingeklebt,Rest.\nnot-wendig (ja/nein) (vor/nach der Digit.),Rest.-\nAufwand gesamt\n(in Std.),Rest.\nerfolgt,Kassette,Schuber,Buch-schuh,Mappe,Um-schlag,SB neu,Anmerkungen (allg.),für Testphase\nvorsehen,Schutzbehältnis empfohlen,Foto für Erheb. Rest. angefertigt (ab August),feuchte-empfind-liches Leder,Material am Rücken/\nEinband lose / eingeris-sen (auch Titelschild),Narben spaltet sich ab,Gelenk(e) \nan/durch-gebro-chen,Bünde gebro-chen (Anzahl),Rücken lose/\nhalb lose,Be-schläge locker,Buch\nschließe fragil,Deckel spaltet sich / Fehlstelle im Deckel,Deckel gebro-chen,Deckel lose / halb lose,Kapital fragil/\nlose,Rest.-Aufwand Einband\n(in Std.),Anmerkungen für die Restaurierung am Einband,Ver-schmutz-ung (Vorsatz / Ränder /\nges. BB),mikro-bieller Befall,Farb-schicht pudert,Buch-block / Seiten verblockt,erste / letzte Lage oder Seiten lose,(halb-) lose Seiten im BB,Heftung zerstört,Risse / Fehl-stellen im Vorsatz,Risse im Text-bereich / an exponier-ter Stelle (z.B. Ecke),Risse am Rand,Fehl-stellen im BB\n(groß),Insekten-fraß (stark),Falten / Knicke,saures /\nbrüchiges Papier,Tinten-/ Farbfraß (akut),Register-marken fragil,Klebe-\nstreifen ablösen,Rest.-Aufwand Buchblock\n(in Std.),Anmerkungen für die Restaurierung am Buchblock
1226,False,1211,L-1521-315488409,1066957789,,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 3, 14 a",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1230,False,1215,L-1519-167114379,998890790,,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 51, 6 a",,,,,,,,,,,,,,,DA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1227,False,1212,L-1522-159340187,995355584,,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 51, 14 a",,,Halbledereinband,,,stark brüchiges Einbandmaterial,,Papierumschlag,Ja,3,,,,,DA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1228,False,1213,L-1520-163974322,997452862,,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 51, 23 b",,,Papier- oder Pappeinband,,,,,Mappe,Ja,3,,Band in Ausstellung,,,DA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1229,False,1214,L-1519-315490586,1066960054,,https://portal.dnb.de/opac.htm?method=simpleSe...,"DA III 51, 31",,,,,,,,,,,,,,,DA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Excel-Dateien mit openpyxl öffnen

In [65]:
wb = openpyxl.load_workbook("Spielwiese_Matchen_III_Drucke_1501-1560.xlsm")
wb.remove(wb["III Drucke 1501-1560"])

/home/vrekon/.cache/pypoetry/virtualenvs/vrekon-Woto_OI2-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [66]:
ws = wb.create_sheet("III Drucke 1501-1560")

In [67]:
df = df.astype("object")
for r in dataframe_to_rows(df[spaltenliste], index=False):
    ws.append(r)

In [68]:
wb.save("iii-merge-test-excel.xlsx")

In [18]:
df.columns = df.columns.str.replace("\n", " ")
df.digi = df.digi.astype("string")
df = df.astype("object")

In [50]:
df.info(True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1332 entries, 1226 to 231
Data columns (total 118 columns):
 #    Column                                                                                              Dtype 
---   ------                                                                                              ----- 
 0    Lfd Nr.                                                                                             object
 1    AKZ                                                                                                 object
 2    IDN                                                                                                 object
 3    Link zum Portal                                                                                     object
 4    Signatur                                                                                            object
 5    Provenienzmerkmal                                                                            

# Anleitung zur Bearbeitung der zusammengefügten Listen

Die Tabelle enthält ganz vorn die neue Spalte "Digi". Steht dort WAHR, wird das Werk digitalisiert, steht FALSCH, wird es nicht digitalisiert.

Zunächst muss überprüft werden, ob Frau Thomschke Daten in eine Zeile erfasst hat, in der eigentlich angebundene Werke oder sonstige untergeordnete Werke stehen. Diese müssen dann in die Zeile des übergeordneten Satzes verschoben werden, z. B. Qd.

Alle mit FALSCH markierten Zeilen, die angebundene Werke und keine Informationen von Frau Thomschke haben, können gelöscht werden.

Alle mit FALSCH markierte Zeilen, die keine IDN und keine AKZ enthalten, aber eine Signatur, wurden von BE "gefunden" für diese Bände muss ein Datensatz angelegt werden. Wenn das passiert ist, wird die IDN und AKZ in die Tabelle kopiert.







